# Dependencies installeren
Dit script heeft `spotipy` nodig. Zorg ervoor dat je 'm geinstalleerd hebt, anders gaat 'ie zeiken!

In [ ]:
## DEPENDENCIES INSTALLEREN

%pip install spotipy

# Imports
Importeert alles wat nodig is om het script te laten werken.

In [ ]:
## IMPORTS

import spotipy
import operator
import random
import spotipy.oauth2
import spotipy.util as util
from datetime import datetime

# Print
print("Import gelukt!")

# Initialiseer `spotipy`
Deze stap initialiseert `spotipy`.
Je moet het volgende doen om `CLIENT_ID` en `CLIENT_SECRET` te verkrijgen:
- Ga naar developer.spotify.com en log in met je Spotify-account.
- Klik op je profiel rechtsboven en ga naar 'Dashboard'.
- Maak een nieuwe app aan, geef 'm een leuke naam en beschrijving. Bij 'Redirect URIs' moet je localhost invullen.
- Bij 'Which API/SDKs are you planning to use?' klik je alleen de 'Web API' aan. Maak de app aan.
- Met de app open klik je op Settings. Hier zie je de Client ID en de Client Secret. Hèhè.

In [ ]:
## INITIALISEER SPOTIPY


CLIENT_ID = "(hier dan je client id)"
CLIENT_SECRET = "(hier dan je client secret)"
CLIENT_USERNAME = "(hier dan je Spotify username)"

token = util.prompt_for_user_token(
    username=CLIENT_USERNAME,
    scope='playlist-modify-public',
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri='http://localhost:8888/callback',
)

spotify = spotipy.Spotify(token)


# Print
print("Spotipy-initialisatie gelukt!")

# Definieer playlist URIs
Hier plak je alle URIs van de ingezonden playlists.  
Om de URI van een playlist te krijgen, rechtsklik je op een playlist in Spotify, ga je naar 'Share' en houd je de Control-toets ingedrukt. Nu zie je 'Copy Playlist URI' staan.

In [ ]:
## DEFINIEER PLAYLIST URIS


playlist_uris = [
    "spotify:playlist:37i9dQZF1Fa6SP6gVRDoDA",
    "spotify:playlist:0xvhuuDdwPIBOgXZWh8FHI",
    # (enzovoort)
]


# Print
print("URI-definitie gelukt!")

# Telling
Deze sectie telt de aanwezigheid van alle liedjes bij elkaar op.  
Je ziet dat ik extra spelregels heb toegevoegd; die mag je natuurlijk weghalen (of meer toevoegen!).  
Opmerking: verschillende uitvoeringen van liedjes tellen als andere liedjes. In mijn jaar heb ik na de telling nog alle tellingen van de verschillende uitvoeringen handmatig bij elkaar opgeteld.

In [ ]:
# TELLING


# Initialiseer telling
counts: dict[str, int] = {}
names: dict[str, str] = {}

# Ga door alle playlists
for playlist_uri in playlist_uris:
    print(f"Processing {playlist_uri}...")

    # Verkrijg alle tracks
    results: dict = spotify.playlist_items(playlist_uri)
    items: list[dict] = results['items']
    while results['next']:
        results = spotify.next(results)
        items.extend(results['items'])

    # Spelregel: filter unieke URI's
    unique_uris: set = set()
    unique_items: list[dict] = []
    for item in items:
        track_uri: str = item['track']['uri']
        
        if track_uri in unique_uris:
            print("Dubbele track verwijderd:", item['track']['name'])
        else:
            unique_items.append(item)

        unique_uris.add(track_uri)

    items = unique_items

    # Spelregel: lijsten zijn max 100 nummers
    items = items[:100]

    # Spelregel: top 5 telt dubbel
    items = items[:5] + items

    # Houd de score bij
    for item in items:
        track_uri: str = item['track']['uri']
        track_name: str = item['track']['name']

        names[track_uri] = track_name

        # Initialiseer telling
        if not (track_uri in counts): counts[track_uri] = 0
        
        counts[track_uri] += 1


# Hack: shuffle de counts
counts_items: list = list(counts.items())
random.shuffle(counts_items)
counts = dict(counts_items)

# Sorteer counts op aantal
top_2_jazzend = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)


# Print
print("Telling gelukt!")

# Schrijf naar CSV
In dit deel wordt de telling geëxporteerd naar een CSV-bestand (en die valt te openen met Excel!). In dit bestand kan je nog kloten, voordat je de uiteindelijke playlist maakt.

In [ ]:
# SCHRIJF NAAR CSV

with open('top_2_jazzend.csv', 'w', encoding='utf8') as f:
    for (uri, count) in top_2_jazzend:
        name = names[uri].replace('"', '')
        
        f.write(uri)
        f.write(';')
        f.write(f'"{name}"')
        f.write(';')
        f.write(str(count))
        f.write('\n')


# Print
print("Schrijven naar CSV gelukt!")

# Maak Spotify playlist
Ah, we zijn er. Met dit stukje code maak je, op basis van de CSV, de playlist automatisch aan in Spotify. Handig!

In [ ]:
# MAAK SPOTIFY PLAYLIST

# Maak een playlist
playlist_name: str = f'[{datetime.now().strftime("%Y-%m-%d %H:%M")}] Top 2-Jazzend'
playlist = spotify.user_playlist_create(user='sjerpsthomas', name=playlist_name, public=False)

# Voeg de nummers toe
with open('top_2_jazzend.csv', 'r', encoding='utf8') as f:
    tracks: list[str] = [line.split(';')[0] for line in f][:100]
    
    spotify.user_playlist_add_tracks(
        user=CLIENT_USERNAME,
        playlist_id=playlist['id'],
        tracks=tracks,
    )


# Print
print("Playlist aanmaken gelukt!")